In [1]:
import os
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import TweetTokenizer
import tensorflow as tf
from keras.utils import np_utils
from keras.models import *
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from tensorflow.python.keras.utils import np_utils
# from tensorflow.python.keras.models import *
# from tensorflow.python.keras.layers import *
# from tensorflow.python.keras.preprocessing.sequence import pad_sequences
# from tensorflow.python.keras import regularizers
# from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
print("imported libs ..!")

imported libs ..!


Using TensorFlow backend.


In [2]:
train = pd.read_csv('tweet-sentiment-extraction-data/train.csv')
test = pd.read_csv('tweet-sentiment-extraction-data/test.csv')

In [3]:
train.dropna(axis=0, how='any', inplace=True)
test.dropna(axis=0, how='any', inplace=True)

In [4]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
text = train['text'].apply(str).values
selected_text = train['selected_text'].apply(str).values
sentiment = train['sentiment'].apply(str).values

In [6]:
text[0].index(selected_text[0])

1

In [7]:
beg = []
end = []
for i in range(len(text)):
    idx = text[i].index(selected_text[i])
    beg.append(text[i][:idx])
    end.append(text[i][idx+len(selected_text[i]):])
    
train['beg'] = beg
train['end'] = end

In [8]:
train.head()

,textID,text,selected_text,sentiment,beg,end
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,,
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,,I will miss you here in San Diego!!!
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is,...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview!,
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,,why couldn`t they put them on the releases we...


In [9]:
tknzr = TweetTokenizer()

In [10]:
## O for outside the sel_text and Y for with in the sel text
xt = []
yt = []

for i in range(len(train)):
    bg = tknzr.tokenize(train.iloc[i]['beg'])
    sel = tknzr.tokenize(train.iloc[i]['selected_text'])
    end = tknzr.tokenize(train.iloc[i]['end'])
    inp = bg + sel + end 
    op = ["O" for i in range(len(bg))] + ["Y" for i in range(len(sel))] + ["O" for i in range(len(end))]
    xt.append(inp)
    yt.append(op)

In [11]:
xt[4]

['Sons',
 'of',
 '*',
 '*',
 '*',
 ',',
 'why',
 'couldn',
 '`',
 't',
 'they',
 'put',
 'them',
 'on',
 'the',
 'releases',
 'we',
 'already',
 'bought']

In [12]:
yt[4]

['Y',
 'Y',
 'Y',
 'Y',
 'Y',
 'Y',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [13]:
words =[]
for i in range(len(xt)):
    words = words + xt[i]

In [14]:
_words = list(set(words))
_words = _words + ['PADDING']

In [15]:
len(_words)

34208

In [16]:
word2idx = {w: i for i,w in enumerate(_words)}

In [17]:
word2idx

{'Apologies': 0,
 'Filipno': 1,
 'ftsk': 2,
 'logins': 3,
 'Happens': 4,
 'meeting': 5,
 'damage': 6,
 'fanclub': 7,
 'Lily': 8,
 'waah': 9,
 'Mattew': 10,
 '_Von_Abe': 11,
 'nomes': 12,
 'australia': 13,
 'alabang': 14,
 'Bose': 15,
 '#Migraines': 16,
 'http://tr.im/kWnX': 17,
 'gots': 18,
 'apathy': 19,
 'seems': 20,
 'Youtube': 21,
 'bedd': 22,
 'sticks': 23,
 'enlgland': 24,
 'nw': 25,
 'Lin-Man': 26,
 'charades': 27,
 'ikea': 28,
 'bk': 29,
 'Bff': 30,
 'creep': 31,
 'Superstar': 32,
 'delhi': 33,
 'http://bit.ly/NgnaR': 34,
 'yeha': 35,
 'arlington': 36,
 'cherry': 37,
 'bklyn': 38,
 '_Ross': 39,
 'pigment': 40,
 'America': 41,
 'splinters': 42,
 'approving': 43,
 'IRON': 44,
 'widget': 45,
 'Gloomy': 46,
 'lovies': 47,
 'Stress-free': 48,
 'hundred': 49,
 'Sore': 50,
 'coFFee': 51,
 'movieee': 52,
 'TWO': 53,
 'turkish': 54,
 'uit': 55,
 'skwl': 56,
 'operational': 57,
 'creche': 58,
 'Teflon': 59,
 'burns': 60,
 'Welcome': 61,
 'FINALY': 62,
 'buying': 63,
 'expert': 64,
 'rly'

In [18]:
tags = ["O", "Y", "PAD"]
tag2idx = {t:i for i,t in enumerate(tags)}
tag2idx

{'O': 0, 'Y': 1, 'PAD': 2}

In [19]:
X = [[word2idx[w] for w in s] for s in xt]

In [20]:
Y = [[tag2idx[t] for t in s] for s in yt]

In [21]:
X[4]

[2517,
 12138,
 7847,
 7847,
 7847,
 27617,
 13108,
 27770,
 20137,
 10169,
 20044,
 28399,
 19906,
 483,
 1306,
 8381,
 9193,
 1868,
 19062]

In [22]:
Y[4]

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
max_len = 0
for a in xt:
    if(max_len < len(a)):
        max_len = len(a)
max_len

66

In [24]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=len(_words) -1)
Y = pad_sequences(maxlen=max_len, sequences=Y, padding="post", value=tag2idx["PAD"])


In [25]:
X[4]

array([ 2517, 12138,  7847,  7847,  7847, 27617, 13108, 27770, 20137,
       10169, 20044, 28399, 19906,   483,  1306,  8381,  9193,  1868,
       19062, 34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207,
       34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207,
       34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207,
       34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207,
       34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207, 34207,
       34207, 34207, 34207], dtype=int32)

In [26]:
Y[4]

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
      dtype=int32)

In [27]:
sentiment_ = ["neutral", "negative", "positive"]
sent2idx = {sen: i for i, sen in enumerate(sentiment_)}

In [28]:
sent2idx

{'neutral': 0, 'negative': 1, 'positive': 2}

In [29]:
sentiment = [sent2idx[w] for w in sentiment]
sentiment[:10]

[0, 1, 1, 1, 1, 0, 2, 0, 0, 2]

In [30]:
sentiment = np_utils.to_categorical(sentiment, num_classes=3)
sentiment[:2]

array([[1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [31]:
X1 = sentiment

In [32]:
y = np.array([ np_utils.to_categorical(i,num_classes=3) for i in Y])

In [33]:
X.shape, X1.shape, y.shape

((27480, 66), (27480, 3), (27480, 66, 3))

In [36]:
!pip install tf2crf

In [61]:
# from tensorflow_addons.text import crf
from tensorflow.keras.layers import Input, Embedding, Bidirectional, GRU, Dense, RepeatVector, TimeDistributed, LSTM
from tensorflow.keras.models import Model
from tf2crf import CRF
input1 = Input(shape=(3,))
fe1 = Dense(128, activation='relu')(input1)
fe2 = RepeatVector(max_len)(fe1)
fe3 = TimeDistributed(Dense(256, activation='relu'))(fe2)

input2 = Input(shape=(max_len,))
se1 = Embedding(len(_words),256, mask_zero=False, input_length=max_len)(input2)
se2 =LSTM(128, return_sequences=True, recurrent_dropout=0.01)(se1)
se3 = Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.01))(se2)

# decoder1 = tf.keras.layers.Add()(se3, fe3])#add([se3, fe3])
decoder1 = tf.keras.layers.Add()([se3, fe3])
output1 = TimeDistributed((Dense(128, activation= 'relu')))(decoder1)
output = TimeDistributed((Dense(64, activation= 'relu')))(output1)
# crf_ = CRF(len(tags))
# out = crf_(output)
# print(out.shape)

In [62]:
model = tf.keras.Model( [input1, input2],  output)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 66)]         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 3)]          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 66, 256)      8757248     input_21[0][0]                   
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 128)          512         input_20[0][0]                   
____________________________________________________________________________________________

In [57]:
model.compile(optimizer="adam", loss=crf_.loss, metrics=[crf_.accuracy])

In [59]:
filepath = "checkpoints"
callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto')
early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')
hist = model.fit(x=[X1,X], y = y, batch_size=128 , epochs=10, callbacks=[callback, early], validation_split=0.1)


Epoch 1/10


InvalidArgumentError:  Incompatible shapes: [128,66] vs. [128]
	 [[node Equal (defined at /home/dlgpu-aus/anaconda3/envs/nlp/lib/python3.7/site-packages/tf2crf/crf.py:91) ]] [Op:__inference_train_function_22225]

Errors may have originated from an input operation.
Input Source operations connected to node Equal:
 ArgMax (defined at /home/dlgpu-aus/anaconda3/envs/nlp/lib/python3.7/site-packages/tf2crf/crf.py:82)	
 ArgMax_2 (defined at /home/dlgpu-aus/anaconda3/envs/nlp/lib/python3.7/site-packages/tf2crf/crf.py:88)

Function call stack:
train_function
